In [1]:
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import html2text
from datetime import datetime

In [2]:
def get_minutes_list(from_date='20050101'):
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')

    for pageIndex in range(1, 31):
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers ={'User-Agent' : user_agent}
        page = requests.get(url, headers=headers)

        try:
            soup = BeautifulSoup(page.content, 'html.parser')
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
            datainfo = soup.find_all('div', class_='col s12 dataInfo')

            for post, data in zip(brdList, datainfo):
                guid = prefix_addr+post.a['href'][:-12]

                desPage = requests.get(guid)
                desSoup = BeautifulSoup(desPage.content, 'html.parser')   
                description = desSoup.find('div', class_='dbData').get_text().strip()
                if description.replace(' ','').find('통화정책방향') >= 0:
                    title = post.find('span', class_='titlesub').get_text().strip()
        
                    mdate = title[title.find(')(') + 2:-1]            
                    if mdate[-1] == '.':
                        mdate = mdate[:-1]
                    mdate = datetime.strptime(mdate, '%Y.%m.%d')

                    if mdate < from_date:
                        break

                    rdate = data.find('span', class_='date').get_text().strip()
                    rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')    

                    get_minutes_file(guid, mdate, rdate)
        except:
            print("get url.content error and pass page{} it".format(pageIndex))

In [4]:
def get_minutes_file(page_addr, mdate, rdate):
    file_header = 'hwp/'
    prefix_addr = "http://bok.or.kr"

    page = requests.get(page_addr)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        links = soup.find('div', class_='addfile').find_all('a')

        for link in links:
            filename = link.get_text()
            filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')

            if filename[-3:] == 'hwp':
                filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d')+ '.hwp'
                file_addr = prefix_addr + link["href"]
                file_res = requests.get(file_addr)
                filepath = file_header + filename

                with open(filepath, 'wb') as f:
                    f.write(file_res.content)

                print('save file name : ' + filename)
    except:
        print("get file failed and pass it")

In [5]:
get_minutes_list()

save file name : 20190830_20190917.hwp
save file name : 20190718_20190806.hwp
save file name : 20190531_20190618.hwp
save file name : 20190418_20190507.hwp
save file name : 20190228_20190319.hwp
save file name : 20190124_20190212.hwp
save file name : 20181130_20181218.hwp
save file name : 20181018_20181106.hwp
save file name : 20180831_20180918.hwp
save file name : 20180712_20180731.hwp
save file name : 20180524_20180612.hwp
save file name : 20180412_20180502.hwp
save file name : 20180227_20180320.hwp
save file name : 20180118_20180206.hwp
save file name : 20171130_20171219.hwp
save file name : 20170831_20170919.hwp
save file name : 20170713_20170801.hwp
save file name : 20170525_20170613.hwp
save file name : 20170413_20170502.hwp
save file name : 20170223_20170314.hwp
save file name : 20170113_20170131.hwp
save file name : 20161215_20170103.hwp
save file name : 20161111_20161129.hwp
save file name : 20161013_20161101.hwp
save file name : 20160909_20160927.hwp
save file name : 20160811

In [6]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [15]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [32]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens+ ngrams

In [33]:
import re
import os
from os import listdir
from os.path import isfile, join

# 폴더내 pdf파일을 txt 파일로 변경
def hwp2txt(source_folder="./hwp/", output_folder="./text/"):
    # 지정 폴더 내 파일 목록 조회 (파일만)
    exefile = 'hwp5txt'
    hwp_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for hwp in hwp_files :
        hwp_filepath = source_folder+hwp
        # pdf 파일을 text로 변환
        filename = output_folder+hwp[:-4]+".txt"
        cmd = f"hwp5txt --output {filename} {hwp_filepath}"
        os.system(cmd)
             
hwp2txt()

In [34]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join
import olefile
    
def preprocessing(source_folder="./hwp/", output_file="./minutes_preprocessed_hwp.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for txt_file in txt_files :
        with open(source_folder + txt_file, "r") as f:
        txt = f.read()
        sections, section_texts = preprocess_minutes(txt)

        ngrams = []
        for section_text in section_texts[1:3] :
            for sentence in section_text :
                ngrams.append(','.join(text2ngram(sentence)))

        #print('@@@'.join(ngrams))
        #break

        df.loc[len(df)] = [txt_file.split('_')[0],
                           txt,
                           '@@@'.join(section_texts[0]),
                           '@@@'.join(section_texts[1]),
                           '@@@'.join(section_texts[2]),
                           '@@@'.join(section_texts[3]),
                           '@@@'.join(section_texts[4]),
                           '@@@'.join(section_texts[5]),
                           len(section_texts[0]),
                           len(section_texts[1]),
                           len(section_texts[2]),
                           len(section_texts[3]),
                           len(section_texts[4]),
                           len(section_texts[5]),
                           '@@@'.join(ngrams)]
    
    df.to_csv(output_file, index=False)
        
preprocessing()

In [35]:
import pandas as pd
df = pd.read_csv("./minutes/minutes_preprocessed.csv")
df[['date','Economic Situation count', 'Foreign Currency count','Financial Markets count','Monetary Policy count','Participant Views count','Government View count']].to_csv("./results/minutes_daily_count.csv", index = False)